## Advanced chunking strategies provided by Amazon Bedrock Knowledge Bases

In this notebook, we will create 3 knowledge bases to provide sample code for the following chunking options supported by Amazon Bedrock Knowledge Bases: 
1. Fixed chunking
2. Semantic chunking
3. Hierarchical chunking
4. Custom chunking using Lambda function


Chunking breaks down the text into smaller segments before embedding. The chunking strategy can't be modified after you create the data source.
As of now, Amazon Bedrock Knowledge Bases only support a few built-in chunking options: no chunking, fixed sized chunking, and default chunking. 

* With Semantic, and Hierarchical chunking features (in addition to the existing options) customers can have more control over how their data is processed and chunked using Lambda function.


We will use a synthetic 10K report as data for a fiticious company called `Octank Financial` to demo the solution.
After creating knowledge bases we will evaluate the results on the same dataset. The focus will be on improving the quality of search results which in turn will improve the accuracy of responses generated by the foundation model. 

## 1. Import the needed libraries
First step is to install the pre-requisites packages.

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ragas==0.1.9 --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import botocore
botocore.__version__

'1.42.2'

In [5]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [6]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '183631345587')

In [7]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name_standard = 'standard-kb'
knowledge_base_name_hierarchical = 'hierarchical-kb'
knowledge_base_name_semantic = 'semantic-kb'
knowledge_base_name_custom = 'custom-chunking-kb'
knowledge_base_description = "Knowledge Base containing complex PDF."
bucket_name = f'{knowledge_base_name_standard}-{suffix}'
intermediate_bucket_name = f'{knowledge_base_name_standard}-intermediate-{suffix}'
lambda_function_name = f'{knowledge_base_name_custom}-lambda-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - Create knowledge bases with fixed chunking strategy
Let's start by creating a [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) to store the restaurant menus. Knowledge Bases allow you to integrate with different vector databases including [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() and [MongoDB Atlas](). For this example, we will integrate the knowledge base with Amazon OpenSearch Serverless. To do so, we will use the helper class `BedrockKnowledgeBase` which will create the knowledge base and all of its pre-requisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

First we will create a knowledge base using fixed chunking strategy followed by hierarchical chunking strategy. 

Parameter values: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [8]:
knowledge_base_standard = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_standard}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-4170119']
buckets_to_check:  ['standard-kb-4170119']
Creating bucket standard-kb-4170119
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_4170119-f) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 04 Dec 2025 17:01:22 '
                                                 'GMT',
                                         'x-amzn-requestid': 'b2bbbe53-4510-4ee6-8eb6-e0e0f38e9663'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'b2bbbe53-451

[2025-12-04 17:02:53,085] p5983 {base.py:258} INFO - PUT https://tdzlsancngpzxr3520x7.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-4170119-f [status:200 request:0.444s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-4170119-f',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 12, 4, 17, 3, 53, 239369, tzinfo=tzlocal()),
  'description': 'Knowledge Base containing complex PDF.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:183631345587:knowledge-base/2IA58XWXPW',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': '2IA58XWXPW',
  'name': 'standard-kb-4170119',
  'roleArn': 'arn:aws:iam::183631345587:role/AmazonBedrockExecutionRoleForKnowledgeBase_4170119-f',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfigur

## 2.1 Upload the dataset to Amazon S3
Now that we have created the knowledge base, let's populate it with the `Octank financial 10K` report dataset. The Knowledge Base data source expects the data to be available on the S3 bucket connected to it and changes on the data can be syncronized to the knowledge base using the `StartIngestionJob` API call. In this example we will use the [boto3 abstraction](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html) of the API, via our helper classe. 

Let's first upload the menu's data available on the `dataset` folder to s3.

In [9]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


Skipping file ../synthetic_dataset/LICENSE
Skipping file ../synthetic_dataset/NOTICE
Skipping file ../synthetic_dataset/README.md
uploading file ../synthetic_dataset/bda.m4v to standard-kb-4170119
uploading file ../synthetic_dataset/octank_financial_10K.pdf to standard-kb-4170119
uploading file ../synthetic_dataset/podcastdemo.mp3 to standard-kb-4170119


Now we start the ingestion job.

In [10]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_standard.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'Q3JTVF97N3',
  'failureReasons': [ '["Encountered error: Ignored 1 files as their file '
                      'format was not supported. [Files: '
                      's3://standard-kb-4170119/podcastdemo.mp3]. Call to '
                      'Customer Source did not succeed.","Encountered error: '
                      'Ignored 1 files as their file format was not supported. '
                      '[Files: s3://standard-kb-4170119/bda.m4v]. Call to '
                      'Customer Source did not succeed."]'],
  'ingestionJobId': 'GUEBGZAGXL',
  'knowledgeBaseId': '2IA58XWXPW',
  'startedAt': datetime.datetime(2025, 12, 4, 17, 4, 27, 503960, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 2,
                  'numberOfDocumentsScanned': 3,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  

Finally we save the Knowledge Base Id to test the solution at a later stage. 

In [11]:
kb_id_standard = knowledge_base_standard.get_knowledge_base_id()

'2IA58XWXPW'


### 2.2 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a foundation model from Bedrock.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

The right response for this query as per ground truth QA pair is: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```

In [12]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019."

In [13]:
time.sleep(20)
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_standard,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

According to the consolidated statements of cash flows for Octank Financial, in the fiscal year ended December 31, 2019:

- Net cash provided by operating activities was $710 million, driven by net income of $700 million and adjustments for non-cash items like depreciation, amortization, and stock-based compensation.

- Net cash used in investing activities was $240 million, likely due to purchases of property, plant, equipment, and marketable securities.

- Net cash provided by financing activities was $350 million, primarily from proceeds of $200 million from issuance of common stock and $300 million from issuance of long-term debt, offset by $50 million used for repurchase of common stock and $100 million for payments of long-term debt. The net increase in cash and cash equivalents for Octank Financial in 2019 was $120 million. Cash and cash equivalents at the beginning of 2019 were $90 million, resulting in cash and cash equivalents of $210 million at the end of 2019.



As you can see, with the retrieve and generate API we get the final response directly, now let's observe the citations for `RetreiveAndGenerate` API. Since, our primary focus on this notebook is to observe the retrieved chunks and citations returned by the model while generating the response. When we provide the relevant context to the foundation model alongwith the query, it will most likely generate the high quality response. 

In [14]:
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

In [15]:
response_standard = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_standard))
citations_rag_print(response_standard)

# of citations or chunks used to generate the response:  4
Chunk 1:  :** | | | | | Proceeds from issuance of common stock | 400 | 300 | 200 | | Repurchase of common stock | ($150) | ($100) | ($50) | | Proceeds from issuance of long-term debt | 500 | 400 | 300 || Payments of long-term debt | ($200) | ($150) | ($100) | | Net cash provided by financing activities | $600 | $450 | $350 | | **Net increase in cash and cash equivalents** | $130 | $140 | $120 | | **Cash and cash equivalents, beginning of year** | $350 | $210 | $90 | | **Cash and cash equivalents, end of year** | $480 | $350 | $210 |     The consolidated statements of cash flows provide information about the cash receipts and payments of Octank Financial during the fiscal years ended December 31, 2021, 2020, and 2019. The statement is divided into three main parts: cash flows from operating activities, investing activities, and financing activities.     In 2021, Octank Financial had net cash provided by operating activities of $

Let's now inspect the source information from the knowledge base with the retrieve API.

#### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chunks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [16]:
def response_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)


In [17]:
response_standard_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_standard, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)

print("# of retrieved results: ", len(response_standard_ret['retrievalResults']))
response_print(response_standard_ret)

# of retrieved results:  5
Chunk 1:  The consolidated financial statements have been approved by the Board of Directors of Octank and authorized for issue on February 15, 2022. They are signed on behalf of the Board by PersonA, Chairperson, and PersonB, Chief Financial Officer.Table for CONSOLIDATED STATEMENTS OF CASH FLOWS in In millions, except share amounts which are reflected in thousands, and per share amounts     **Octank Financial Consolidated Statements of Cash Flows**     The following table presents the consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2021, 2020, and 2019. All amounts are in millions, except share amounts which are reflected in thousands, and per share amounts.     | (In millions) | 2021 | 2020 | 2019 | | --- | --- | --- | --- | | **Cash flows from operating activities:** | | | | | Net income | $1,200 | $900 | $700 | | Adjustments to reconcile net income to net cash provided by operating activities: | | | | | D

As you can notice, that with `fixed chunking` we get 5 retrieved results as requested in the API using `semantic similarity` which is the default for `Retrieve API`. Let's now use `hierarchical chunking` strategy and inspect the retrieved results using `RetrieveAndGenerate` API as well as `Retrieve` API. 

## 3. Create knowledge bases with hierarchical chunking strategy

**Concept**

Hierarchical chunking: Organizes your data into a hierarchical structure, allowing for more granular and efficient retrieval based on the inherent relationships within your data. Organizing your data into a hierarchical structure enables your RAG workflow to efficiently navigate and retrieve information from complex, nested datasets.
After the documents are parsed, the first step is to chunk the documents based on the parent and child chunking size. The chunks are then organized into a hierarchical structure, where parent chunk (higher level) represents larger chunks (for example, documents or sections), and child chunks (lower level) represent smaller chunks (for example, paragraphs or sentences). The relationship between the parent and child chunks are maintained. This hierarchical structure allows for efficient retrieval and navigation of the corpus.

**Benefits:**

- Efficient retrieval: The hierarchical structure allows faster and more targeted retrieval of relevant information; first by performing semantic search on the child chunk and then returning the parent chunk during retrieval. By replacing the children chunks with the parent chunk, we provide large and comprehensive context to the FM.
- Context preservation: Organizing the corpus in a hierarchical manner helps preserve the contextual relationships between chunks, which can be beneficial for generating coherent and contextually relevant text.

><br>          
>Note:
>In hierarchical chunking, parent chunks are returned and search is performed on children chunks, therefore, you might see less number of search results returned as one parent can have multiple children.
> <br></br>       

Hierarchical chunking is best suited for complex documents that have a nested or hierarchical structure, such as technical manuals, legal documents, or academic papers with complex formatting and nested tables.

**Parameter values:** 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [18]:
knowledge_base_hierarchical = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_hierarchical}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "HIERARCHICAL", 
    suffix = f'{suffix}-h'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-4170119']
buckets_to_check:  ['standard-kb-4170119']
Bucket standard-kb-4170119 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_4170119-h) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 04 Dec 2025 17:05:44 '
                                                 'GMT',
                                         'x-amzn-requestid': '95097843-666d-41f8-918a-9dee2263161d'},
                        'HTTPStatusCode': 200,
                        'Re

[2025-12-04 17:07:15,267] p5983 {base.py:258} INFO - PUT https://ay2gao3tbtwu9ksnsmke.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-4170119-h [status:200 request:0.428s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-4170119-h',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is HIERARCHICAL
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 12, 4, 17, 8, 15, 424973, tzinfo=tzlocal()),
  'description': 'Knowledge Base containing complex PDF.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:183631345587:knowledge-base/INM1P0ZZW7',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'INM1P0ZZW7',
  'name': 'hierarchical-kb-4170119',
  'roleArn': 'arn:aws:iam::183631345587:role/AmazonBedrockExecutionRoleForKnowledgeBase_4170119-h',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessCo

Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [19]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_hierarchical.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'QCW0BQ5UC9',
  'failureReasons': [ '["Encountered error: Ignored 1 files as their file '
                      'format was not supported. [Files: '
                      's3://standard-kb-4170119/podcastdemo.mp3]. Call to '
                      'Customer Source did not succeed.","Encountered error: '
                      'Ignored 1 files as their file format was not supported. '
                      '[Files: s3://standard-kb-4170119/bda.m4v]. Call to '
                      'Customer Source did not succeed."]'],
  'ingestionJobId': 'N30WKFTHDU',
  'knowledgeBaseId': 'INM1P0ZZW7',
  'startedAt': datetime.datetime(2025, 12, 4, 17, 8, 49, 183554, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 2,
                  'numberOfDocumentsScanned': 3,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  

Save the knowledge base id for further testing. 

In [20]:
kb_id_hierarchical = knowledge_base_hierarchical.get_knowledge_base_id()

'INM1P0ZZW7'


### 3.1 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a foundation model from Bedrock.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

The right response for this query as per ground truth QA pair is: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```

In [21]:
time.sleep(20)
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_hierarchical,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

According to the consolidated statements of cash flows for Octank Financial, in the fiscal year ended December 31, 2019, the company had:

Net cash provided by operating activities: $710 million
Net cash used in investing activities: $240 million
Net cash provided by financing activities: $350 million

This resulted in a net increase in cash and cash equivalents of $120 million for the year. The cash and cash equivalents at the beginning of 2019 were $90 million, and at the end of the year were $210 million.



As you can see, with the `RetreiveAndGenerate` API we get the final response directly, now let's observe the citations for `RetreiveAndGenerate` API. Since, our primary focus on this notebook is to observe the retrieved chunks and citations returned by the model while generating the response. When we provide the relevant context to the foundation model alongwith the query, it will most likely generate the high quality response. 

In [22]:
response_hierarchical = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_hierarchical))
citations_rag_print(response_hierarchical)

# of citations or chunks used to generate the response:  1
Chunk 1:  Share Capital** | | | | - Common Shares, $0.01 par value; outstanding: | | | | PersonA, 10,000 shares | 100 | 100 | | PersonB, 20,000 shares | 200 | 200 | | PersonC, 30,000 shares | 300 | 300 | | - Total Common Shares Outstanding (thousands) | 60,000 | 60,000 | | - Common Shares, $0.01 par value; authorized but unissued: | 1,000,000 | 1,000,000 | | **II. Redeemable Non-Controlling Interests** | | | | - Redeemable preferred shares, $0.01 par value; | | | | outstanding: | | | | Investment Group X, 5,000 shares | 50 | 50 | | Investment Group Y, 10,000 shares | 100 | 100 | | - Total Redeemable Preferred Shares Outstanding (thousands)| 15,000 | 15,000 | | - Redeemable preferred shares, $0.01 par value; | | | | authorized but unissued: | | | | Investment Group Z, 5,000 shares | 50 | 50 | | **III. Accumulated Other Comprehensive Income (Loss)** | (10) | 20 | | **IV. Retained Earnings** | 1,200 | 1,000 | | **V. Total Octank F

Let's now retrieve the source information from the knowledge base with the retrieve API.

#### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [23]:
response_hierarchical_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_hierarchical, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)

print("# of retrieved results: ", len(response_hierarchical_ret['retrievalResults']))
response_print(response_hierarchical_ret)

# of retrieved results:  3
Chunk 1:  Share Capital** | | | | - Common Shares, $0.01 par value; outstanding: | | | | PersonA, 10,000 shares | 100 | 100 | | PersonB, 20,000 shares | 200 | 200 | | PersonC, 30,000 shares | 300 | 300 | | - Total Common Shares Outstanding (thousands) | 60,000 | 60,000 | | - Common Shares, $0.01 par value; authorized but unissued: | 1,000,000 | 1,000,000 | | **II. Redeemable Non-Controlling Interests** | | | | - Redeemable preferred shares, $0.01 par value; | | | | outstanding: | | | | Investment Group X, 5,000 shares | 50 | 50 | | Investment Group Y, 10,000 shares | 100 | 100 | | - Total Redeemable Preferred Shares Outstanding (thousands)| 15,000 | 15,000 | | - Redeemable preferred shares, $0.01 par value; | | | | authorized but unissued: | | | | Investment Group Z, 5,000 shares | 50 | 50 | | **III. Accumulated Other Comprehensive Income (Loss)** | (10) | 20 | | **IV. Retained Earnings** | 1,200 | 1,000 | | **V. Total Octank Financial Equity** | $1,310 | $1,

><br>
> Note:
>As you can see in the above response, that the `retrieve` API returned only 3 search results or chunks although 5 were passed in the request. The reason is that with `hiearchical` chunking, parent chunks are returned by the API whereas search is performed on `children chunks` and one `parent chunk` can have multiple `children chunks`. Therefore, response returned only 3 chunks while the search was performed on 5 `children chunks`.
><br></br>

## 4. Create knowledge bases with semantic chunking strategy

**Concept**

Semantic chunking analyzes the relationships within a text and divides it into meaningful and complete chunks, which are derived based on the semantic similarity calculated by the embedding model. This approach preserves the information’s integrity during retrieval, helping to ensure accurate and contextually appropriate results.
Amazon Bedrock Knowledge Bases first divides documents into chunks based on the specified token size. Embeddings are created for each chunk, and similar chunks in the embedding space are combined based on the similarity threshold and buffer size, forming new chunks. Consequently, the chunk size can vary across chunks.

**Benefits**

- By focusing on the text’s meaning and context, semantic chunking significantly improves the quality of retrieval. It should be used in scenarios where maintaining the semantic integrity of the text is crucial.

- Although this method is more computationally intensive than fixed-size chunking, it can be beneficial for chunking documents where contextual boundaries aren’t clear—for example, legal documents or technical manuals.[[1]](#https://www.mongodb.com/developer/products/atlas/choosing-chunking-strategy-rag/)

**Parameter values:**
 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [24]:
knowledge_base_semantic = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_semantic}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "SEMANTIC", 
    suffix = f'{suffix}-s'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-4170119']
buckets_to_check:  ['standard-kb-4170119']
Bucket standard-kb-4170119 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_4170119-s) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 04 Dec 2025 17:10:08 '
                                                 'GMT',
                                         'x-amzn-requestid': 'be652808-d1fa-4cc1-85c3-564e901a294e'},
                        'HTTPStatusCode': 200,
                        'Re

[2025-12-04 17:11:39,130] p5983 {base.py:258} INFO - PUT https://sp92hrrkw1ol7auv2dj8.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-4170119-s [status:200 request:0.390s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-4170119-s',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is SEMANTIC
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 12, 4, 17, 12, 39, 244765, tzinfo=tzlocal()),
  'description': 'Knowledge Base containing complex PDF.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:183631345587:knowledge-base/XRYDWXOYOA',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'XRYDWXOYOA',
  'name': 'semantic-kb-4170119',
  'roleArn': 'arn:aws:iam::183631345587:role/AmazonBedrockExecutionRoleForKnowledgeBase_4170119-s',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfigura

Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [25]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_semantic.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'JEECZLMSD5',
  'failureReasons': [ '["Encountered error: Ignored 1 files as their file '
                      'format was not supported. [Files: '
                      's3://standard-kb-4170119/podcastdemo.mp3]. Call to '
                      'Customer Source did not succeed.","Encountered error: '
                      'Ignored 1 files as their file format was not supported. '
                      '[Files: s3://standard-kb-4170119/bda.m4v]. Call to '
                      'Customer Source did not succeed."]'],
  'ingestionJobId': 'JDOMU3JHRU',
  'knowledgeBaseId': 'XRYDWXOYOA',
  'startedAt': datetime.datetime(2025, 12, 4, 17, 13, 12, 703343, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 2,
                  'numberOfDocumentsScanned': 3,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                 

In [26]:
kb_id_semantic = knowledge_base_semantic.get_knowledge_base_id()

'XRYDWXOYOA'


### 4.1 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a foundation model from Bedrock.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

The right response for this query as per ground truth QA pair is: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```

In [27]:
time.sleep(20)

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_semantic,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The consolidated statements of cash flows for Octank Financial for the fiscal year ended December 31, 2019 show the following:

Cash flows from operating activities: $110 million
Cash flows used in investing activities: $(240) million, including $(60) million for purchases of property and equipment and $20 million from maturing marketable securities.
Cash flows from financing activities: $350 million, including $200 million from issuance of common stock, $(50) million for repurchase of common stock, $300 million from issuance of long-term debt, and $(100) million for payments of long-term debt.

The net increase in cash and cash equivalents for 2019 was $120 million. Cash and cash equivalents at the beginning of the year were $90 million and at the end of the year were $210 million.



As you can see, with the `RetreiveAndGenerate` API we get the final response directly, now let's observe the citations for `RetreiveAndGenerate` API. Since, our primary focus on this notebook is to observe the retrieved chunks and citations returned by the model while generating the response. When we provide the relevant context to the foundation model alongwith the query, it will most likely generate the high quality response. 

In [28]:
response_semantic = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_semantic))
citations_rag_print(response_semantic)

# of citations or chunks used to generate the response:  2
Chunk 1:  100) | ($80) | ($60) | | Maturing marketable securities | 40 | 30 | 20 | | Net cash used in investing activities | ($360) | ($290) | ($240) | | **Cash flows from financing activities:** | | | | | Proceeds from issuance of common stock | 400 | 300 | 200 | | Repurchase of common stock | ($150) | ($100) | ($50) | | Proceeds from issuance of long-term debt | 500 | 400 | 300 || Payments of long-term debt | ($200) | ($150) | ($100) | | Net cash provided by financing activities | $600 | $450 | $350 | | **Net increase in cash and cash equivalents** | $130 | $140 | $120 | | **Cash and cash equivalents, beginning of year** | $350 | $210 | $90 | | **Cash and cash equivalents, end of year** | $480 | $350 | $210 | 
 
 The consolidated statements of cash flows provide information about the cash receipts and payments of Octank Financial during the fiscal years ended December 31, 2021, 2020, and 2019. The statement is divided into th

Let's now retrieve the source information from the knowledge base with the retrieve API.

#### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [29]:
response_semantic_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_semantic, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)
print("# of citations or chunks used to generate the response: ", len(response_semantic_ret['retrievalResults']))
response_print(response_semantic_ret)

# of citations or chunks used to generate the response:  5
Chunk 1:  This table offers a clear and concise overview of Octank Financial's equity structure, allowing stakeholders to understand the company's financial position and performance.Table for CONSOLIDATED STATEMENTS OF CASH FLOWS in In millions, except share amounts which are reflected in thousands, and per share amounts 
 
 **Octank Financial Consolidated Statements of Cash Flows** 
 
 The following table presents the consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2021, 2020, and 2019. All amounts are in millions, except share amounts which are reflected in thousands, and per share amounts

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-4170119/octank_financial_10K.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.6991188

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-4170119/octank_financial_10K.pdf', 'x-amz-bedrock-kb-source-file-modality': 'TEXT', 

## 5. Custom chunking option using Lambda Functions
When creating an Knowledge Bases (KB) for Amazon Bedrock, you can connect a Lambda function to specify your custom chunking logic. During ingestion, if lambda function is provided, Knowledge Bases, will run the lambda function, and store the input and output values in the intermediate s3 bucket provided.

> <br>
> Note: Lambda function with KB can be used for adding custom chunking logic as well processing your chunks for example, adding chunk level metadata. In this example we are focusing on using Lambda function for custom chunking logic.
> <br></br>

### 5.1 Create the Lambda Function

We will now create a lambda function which will have code for custom chunking. To do so we will:

1. Create the `lambda_function.py` file which contains the logic for custom chunking.
2. Create the IAM role for our Lambda function.
3. Create the lambda function with the required permissions.

#### Create the function code
 Let's create the lambda function tha implements the functions for `reading your file from intermediate bucket`, `process the contents with custom chunking logic` and `write the output back to s3 bucket`. 

In [30]:
%%writefile lambda_function.py
import json
from abc import abstractmethod, ABC
from typing import List
from urllib.parse import urlparse
import boto3
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

class Chunker(ABC):
    @abstractmethod
    def chunk(self, text: str) -> List[str]:
        raise NotImplementedError()
        
class SimpleChunker(Chunker):
    def chunk(self, text: str) -> List[str]:
        words = text.split()
        return [' '.join(words[i:i+100]) for i in range(0, len(words), 100)]

def lambda_handler(event, context):
    logger.debug('input={}'.format(json.dumps(event)))
    s3 = boto3.client('s3')

    # Extract relevant information from the input event
    input_files = event.get('inputFiles')
    input_bucket =  event.get('bucketName')

    
    if not all([input_files, input_bucket]):
        raise ValueError("Missing required input parameters")
    
    output_files = []
    chunker = SimpleChunker()

    for input_file in input_files:
        content_batches = input_file.get('contentBatches', [])
        file_metadata = input_file.get('fileMetadata', {})
        original_file_location = input_file.get('originalFileLocation', {})

        processed_batches = []
        
        for batch in content_batches:
            input_key = batch.get('key')

            if not input_key:
                raise ValueError("Missing uri in content batch")
            
            # Read file from S3
            file_content = read_s3_file(s3, input_bucket, input_key)
            
            # Process content (chunking)
            chunked_content = process_content(file_content, chunker)
            
            output_key = f"Output/{input_key}"
            
            # Write processed content back to S3
            write_to_s3(s3, input_bucket, output_key, chunked_content)
            
            # Add processed batch information
            processed_batches.append({
                'key': output_key
            })
        
        # Prepare output file information
        output_file = {
            'originalFileLocation': original_file_location,
            'fileMetadata': file_metadata,
            'contentBatches': processed_batches
        }
        output_files.append(output_file)
    
    result = {'outputFiles': output_files}
    
    return result
    

def read_s3_file(s3_client, bucket, key):
    response = s3_client.get_object(Bucket=bucket, Key=key)
    return json.loads(response['Body'].read().decode('utf-8'))

def write_to_s3(s3_client, bucket, key, content):
    s3_client.put_object(Bucket=bucket, Key=key, Body=json.dumps(content))    

def process_content(file_content: dict, chunker: Chunker) -> dict:
    chunked_content = {
        'fileContents': []
    }
    
    for content in file_content.get('fileContents', []):
        content_body = content.get('contentBody', '')
        content_type = content.get('contentType', '')
        content_metadata = content.get('contentMetadata', {})
        
        words = content['contentBody']
        chunks = chunker.chunk(words)
        
        for chunk in chunks:
            chunked_content['fileContents'].append({
                'contentType': content_type,
                'contentMetadata': content_metadata,
                'contentBody': chunk
            })
    
    return chunked_content

Writing lambda_function.py


The standard chunking strategies values provided by knowledge bases are following: 

**Parameter values:**
 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

For implementing our custom logic, we have included an option in the `knowledge_base.py` class for passing a value of `CUSTOM"`. 
If you pass the chunking strategy as `CUSTOM` in this class, it will do the following: 

1. It select the `chunkingStrategy` as `NONE`. 
2. It will add `customTransformationConfiguration` to the `vectorIngestionConfiguration` as follows: 

```
{
...
   "vectorIngestionConfiguration": {
    "customTransformationConfiguration": { 
         "intermediateStorage": { 
            "s3Location": { 
               "uri": "string"
            }
         },
         "transformations": [
            {
               "transformationFunction": {
                  "lambdaConfiguration": {
                     "lambdaArn": "string"
                  }
               },
               "stepToApply": "string" // enum of POST_CHUNKING
            }
         ]
      },
      "chunkingConfiguration": {
         "chunkingStrategy": "NONE"
         ...
   }
}

```


In [31]:
knowledge_base_custom = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_custom}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    lambda_function_name=lambda_function_name,
    intermediate_bucket_name=intermediate_bucket_name, 
    chunking_strategy = "CUSTOM", 
    suffix = f'{suffix}-c'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-4170119', 'standard-kb-intermediate-4170119']
buckets_to_check:  ['standard-kb-4170119', 'standard-kb-intermediate-4170119']
Bucket standard-kb-4170119 already exists - retrieving it!
Creating bucket standard-kb-intermediate-4170119
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_4170119-c) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 04 Dec 2025 17:15:12 '
                                                 'GMT',
                                         'x-amzn-reques

[2025-12-04 17:16:43,108] p5983 {base.py:258} INFO - PUT https://dz1mnun30ozv62qu5axb.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-4170119-c [status:200 request:0.392s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-4170119-c',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Creating lambda function... as chunking strategy is CUSTOM
{'ResponseMetadata': {'RequestId': '5702653c-16f0-48dc-9ec0-f39cc9acefa9', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Thu, 04 Dec 2025 17:17:54 GMT', 'content-type': 'application/json', 'content-length': '1491', 'connection': 'keep-alive', 'x-amzn-requestid': '5702653c-16f0-48dc-9ec0-f39cc9acefa9'}, 'RetryAttempts': 0}, 'FunctionName': 'custom-chunking-kb-lambda-4170119', 'FunctionArn': 'arn:aws:lambda:us-west-2:183631345587:function:custom-chunking-kb-lambda-4170119', 'Runtime': 'python3.12', 'Role': 'arn:aws:iam::183631345587:role/custom-chunking-kb-4170119-lambda-role-4170119-c', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 3465, 'Description': '', 'Timeout': 60, 'MemorySize': 128, 'LastModified': '2025-12-04T17:1

Now start the ingestion job. 

In [32]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_custom.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'SU4Q84WECX',
  'failureReasons': [ '["Encountered error: Ignored 1 files as their file '
                      'format was not supported. [Files: '
                      's3://standard-kb-4170119/podcastdemo.mp3]. Call to '
                      'Customer Source did not succeed.","Encountered error: '
                      'Ignored 1 files as their file format was not supported. '
                      '[Files: s3://standard-kb-4170119/bda.m4v]. Call to '
                      'Customer Source did not succeed."]'],
  'ingestionJobId': '6BFIKIUAE1',
  'knowledgeBaseId': 'SFRHWPIW89',
  'startedAt': datetime.datetime(2025, 12, 4, 17, 18, 27, 142076, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 2,
                  'numberOfDocumentsScanned': 3,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                 

In [33]:
kb_id_custom = knowledge_base_custom.get_knowledge_base_id()

'SFRHWPIW89'


### 5.2 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a foundation model from Bedrock.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

The right response for this query as per ground truth QA pair is: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```

In [34]:
time.sleep(10)

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_custom,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

According to the consolidated statements of cash flows for Octank Financial, in the fiscal year ended December 31, 2019:

- Net cash provided by operating activities was $710 million.
- Cash and cash equivalents at the beginning of the year was $90 million.
- Cash and cash equivalents at the end of the year was $210 million. The consolidated statements of cash flows provide information about Octank Financial's cash receipts and payments during 2019. It is divided into three main sections: cash flows from operating activities, investing activities, and financing activities. The cash flows demonstrate the company's ability to generate cash from operations and finance its growth and investments.



As you can see, with the `RetreiveAndGenerate` API we get the final response directly, now let's observe the citations for `RetreiveAndGenerate` API. Since, our primary focus on this notebook is to observe the retrieved chunks and citations returned by the model while generating the response. When we provide the relevant context to the foundation model alongwith the query, it will most likely generate the high quality response. 

In [35]:
response_custom = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_custom))
citations_rag_print(response_custom)

# of citations or chunks used to generate the response:  2
Chunk 1:  activities | $600 | $450 | $350 | | **Net increase in cash and cash equivalents** | $130 | $140 | $120 | | **Cash and cash equivalents, beginning of year** | $350 | $210 | $90 | | **Cash and cash equivalents, end of year** | $480 | $350 | $210 | The consolidated statements of cash flows provide information about the cash receipts and payments of Octank Financial during the fiscal years ended December 31, 2021, 2020, and 2019. The statement is divided into three main parts: cash flows from operating activities, investing activities, and financing activities. In

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-4170119/octank_financial_10K.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-4170119/octank_financial_10K.pdf', 'x-amz-bedrock-kb-document-page-number': 1.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AjZtf6poBAYJI6zCymgUF', 'x-amz-bedrock-kb-data-source-id': 'SU4Q8

Let's now retrieve the source information from the knowledge base with the retrieve API.

#### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [36]:
response_custom_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_custom, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)
print("# of citations or chunks used to generate the response: ", len(response_custom_ret['retrievalResults']))
response_print(response_custom_ret)

# of citations or chunks used to generate the response:  5
Chunk 1:  and concise overview of Octank Financial's equity structure, allowing stakeholders to understand the company's financial position and performance.Table for CONSOLIDATED STATEMENTS OF CASH FLOWS in In millions, except share amounts which are reflected in thousands, and per share amounts **Octank Financial Consolidated Statements of Cash Flows** The following table presents the consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2021, 2020, and 2019. All amounts are in millions, except share amounts which are reflected in thousands, and per share amounts. | (In millions) | 2021 | 2020 | 2019 | | --- |

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-4170119/octank_financial_10K.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.76009154

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-4170119/octank_financial_10K.pdf', 'x-amz-bedrock-kb-document-page-nu

In all cases, while evaluating one query, we got the correct response. However, when you are building a RAG application, you need to evaluate with large number of Questions and Answers to figure out the accuracy improvements. In the next step, we will use RAG Assessment (RAGAS) open source framework to evaluate the responses on `your dataset` for the metrics related to evaluating the quality of the context or search results.
We will focus only on 2 metrics: 

1. Context recall
2. Context relevancy

## 6. Evaluating search results using RAG Assessment (RAGAS) framework on your dataset
You can use RAGAS framework to evaluate your results for each chunking strategy. This approach can help you provide factual guidance on which chunking strategy to use for your dataset. 

Ideally, you should consider optimizing on other parameters as well for example in case of heirarchical chunking you should consider trying different sizes for parent chunk or child chunk. 

Below approach will provide you heuristics as to which strategy could be used based on the default parameters recommended by Amazon Bedrock Knowledge Bases. 

In [37]:
print("Semantic: ", kb_id_semantic)
print("Standard: ", kb_id_standard)
print("Hierarchical: ", kb_id_hierarchical)
print("Custom chunking: ", kb_id_custom)

Semantic:  XRYDWXOYOA
Standard:  2IA58XWXPW
Hierarchical:  INM1P0ZZW7
Custom chunking:  SFRHWPIW89


#### Evaluation
In this section we will utilize RAGAS for evaluating search results using following metrics:
1. **Context Recall:** Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.

2. **Context relevancy:** This metric gauges the relevancy of the retrieved context, calculated based on both the question and contexts. The values fall within the range of (0, 1), with higher values indicating better relevancy.

In [38]:
from utils.evaluation import KnowledgeBasesEvaluations

from ragas.metrics import (
    context_recall,
    context_precision,
    )

metrics = [context_recall,
           context_precision
           ]

MODEL_ID_EVAL = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_ID_GEN = "anthropic.claude-3-haiku-20240307-v1:0"

questions = [
        "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.",
]
ground_truths = [
    "The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:\
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.\
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).\
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. \
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.",
]
kb_evaluate_standard = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, 
                        KB_ID=kb_id_standard,
                        )

kb_evaluate_hierarchical = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, KB_ID=kb_id_hierarchical)

kb_evaluate_semantic = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, KB_ID=kb_id_semantic)

kb_evaluate_custom = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, KB_ID=kb_id_custom)

/opt/conda/lib/python3.12/site-packages/ragas/metrics/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
/opt/conda/lib/python3.12/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to impo

In [39]:
results_heirarchical = kb_evaluate_hierarchical.evaluate()
results_standard = kb_evaluate_standard.evaluate()
results_semantic = kb_evaluate_semantic.evaluate()
results_custoom = kb_evaluate_custom.evaluate()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
import pandas as pd
pd.options.display.max_colwidth = 800
print("Fixed Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_standard["context_recall"].mean())
print("Average context_relevancy: ", results_standard["context_precision"].mean(), "\n")

print("Hierarchical Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_heirarchical["context_recall"].mean())
print("Average context_relevancy: ", results_heirarchical["context_precision"].mean(), "\n")

print("Semantic Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_semantic["context_recall"].mean())
print("Average context_relevancy: ", results_semantic["context_precision"].mean(), "\n")

print("Custom Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_custoom["context_recall"].mean())
print("Average context_relevancy: ", results_custoom["context_precision"].mean())


Fixed Chunking Evaluation for synthetic 10K report
--------------------------------------------------------------------
Average context_recall:  0.8
Average context_relevancy:  0.9333333333177777 

Hierarchical Chunking Evaluation for synthetic 10K report
--------------------------------------------------------------------
Average context_recall:  1.0
Average context_relevancy:  0.99999999995 

Semantic Chunking Evaluation for synthetic 10K report
--------------------------------------------------------------------
Average context_recall:  0.8
Average context_relevancy:  0.7888888888731111 

Custom Chunking Evaluation for synthetic 10K report
--------------------------------------------------------------------
Average context_recall:  0.6
Average context_relevancy:  0.499999999975


In [41]:
print("===============================Knowledge base with fixed chunking==============================\n")
knowledge_base_standard.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
print("===============================Knowledge base with hierarchical chunking==============================\n")
knowledge_base_hierarchical.delete_kb(delete_s3_bucket=False,delete_iam_roles_and_policies=True)
print("===============================Knowledge base with semantic chunking==============================\n")
knowledge_base_semantic.delete_kb(delete_s3_bucket=False,delete_iam_roles_and_policies=True)
print("===============================Knowledge base with custom chunking==============================\n")
knowledge_base_custom.delete_kb(delete_s3_bucket=True,delete_iam_roles_and_policies=True, delete_lambda_function = True)

===============================Knowledge base with fixed chunking==============================

Deleted data source Q3JTVF97N3
======== Knowledge base and all data sources deleted =========
Found bucket standard-kb-4170119
Deleted all objects in bucket standard-kb-4170119
Deleted bucket standard-kb-4170119
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_4170119-f
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_4170119-f========
 [{'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_4170119-f', 'PolicyArn': 'arn:aws:iam::183631345587:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_4170119-f'}, {'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_4170119-f', 'PolicyArn': 'arn:aws:iam::183631345587:policy/AmazonBedrockOSSPolicyForKnowledgeBase_4170119-f'}, {'PolicyName': 'AmazonBedrockCloudWatchPolicyForKnowledgeBase_4170119-f', 'PolicyArn': 'arn:aws:iam::183631345587:poli